In [52]:
"""
linuxrouter.py: Example network with Linux IP router
This example converts a Node into a router using IP forwarding
already built into Linux.
The example topology creates a router and three IP subnets:
    - 192.168.1.0/24 (r0-eth1, IP: 192.168.1.1)
    - 172.16.0.0/12 (r0-eth2, IP: 172.16.0.1)
    - 10.0.0.0/8 (r0-eth3, IP: 10.0.0.1)
Each subnet consists of a single host connected to
a single switch:
    r0-eth1 - s1-eth1 - h1-eth0 (IP: 192.168.1.100)
    r0-eth2 - s2-eth1 - h2-eth0 (IP: 172.16.0.100)
    r0-eth3 - s3-eth1 - h3-eth0 (IP: 10.0.0.100)
The example relies on default routing entries that are
automatically created for each router interface, as well
as 'defaultRoute' parameters for the host interfaces.
Additional routes may be added to the router or hosts by
executing 'ip route' or 'route' commands on the router or hosts.
"""

"\nlinuxrouter.py: Example network with Linux IP router\nThis example converts a Node into a router using IP forwarding\nalready built into Linux.\nThe example topology creates a router and three IP subnets:\n    - 192.168.1.0/24 (r0-eth1, IP: 192.168.1.1)\n    - 172.16.0.0/12 (r0-eth2, IP: 172.16.0.1)\n    - 10.0.0.0/8 (r0-eth3, IP: 10.0.0.1)\nEach subnet consists of a single host connected to\na single switch:\n    r0-eth1 - s1-eth1 - h1-eth0 (IP: 192.168.1.100)\n    r0-eth2 - s2-eth1 - h2-eth0 (IP: 172.16.0.100)\n    r0-eth3 - s3-eth1 - h3-eth0 (IP: 10.0.0.100)\nThe example relies on default routing entries that are\nautomatically created for each router interface, as well\nas 'defaultRoute' parameters for the host interfaces.\nAdditional routes may be added to the router or hosts by\nexecuting 'ip route' or 'route' commands on the router or hosts.\n"

In [53]:
from mininet.topo import Topo
from mininet.net import Mininet
from mininet.node import Node
from mininet.log import setLogLevel, info
from mininet.cli import CLI

In [54]:
class LinuxRouter( Node ):
    "A Node with IP forwarding enabled."

    def config( self, **params ):
        super( LinuxRouter, self).config( **params )
        # Enable forwarding on the router
        self.cmd( 'sysctl net.ipv4.ip_forward=1' )

    def terminate( self ):
        self.cmd( 'sysctl net.ipv4.ip_forward=0' )
        super( LinuxRouter, self ).terminate()

In [55]:
class NetworkTopo( Topo ):
    "A LinuxRouter connecting three IP subnets"

    def build( self, **_opts ):

        defaultIP = '192.168.1.1/24'  # IP address for r0-eth1
        defaultIP_2 = '192.168.2.1/24'  # IP address for r0-eth1
        router = self.addNode( 'r0', cls=LinuxRouter, ip=defaultIP )
        router_2 = self.addNode( 'r1', cls=LinuxRouter, ip=defaultIP_2)

        s1, s2, s3 = [ self.addSwitch( s ) for s in ( 's1', 's2', 's3' ) ]
        
#         s21, s22, s23 = [self.addSwitch( s ) for s in ('s1', 's2', 's3' ) ]
        
#         self.addLink( s21, router, intfName2='r0-eth1',
#                       params2={ 'ip' : defaultIP_2 } )  # for clarity
        
#         self.addLink( s22, router, intfName2='r0-eth2',
#                       params2={ 'ip' : '172.16.0.1/12' } )
        
#         self.addLink( s23, router, intfName2='r0-eth3',
#                       params2={ 'ip' : '10.0.0.1/8' } )
        
        
        self.addLink( s1, router, intfName2='r0-eth1',
                      params2={ 'ip' : defaultIP } )  # for clarity
#         self.addLink( router, router_2, intfName2='r0-eth2',
#                       params2={ 'ip' : defaultIP_2 } )  # for clarity
#         self.addLink( router, router_2, intfName2='r0-eth2',
#                       params2={ 'ip' : defaultIP } )  # for clarity
#         self.addLink( s2, router, intfName2='r0-eth2',
#                       params2={ 'ip' : '172.16.0.1/12' } )
#         self.addLink( s3, router, intfName2='r0-eth3',
#                       params2={ 'ip' : '10.0.0.1/8' } )

#         h1 = self.addHost( 'h1', ip='192.168.1.100/24',
#                            defaultRoute='via 192.168.1.1' )
#         h2 = self.addHost( 'h2', ip='172.16.0.100/12',
#                            defaultRoute='via 172.16.0.1' )
#         h3 = self.addHost( 'h3', ip='10.0.0.100/8',
#                            defaultRoute='via 10.0.0.1' )

#         for h, s in [ (h1, s1), (h2, s2), (h3, s3) ]:
#             self.addLink( h, s )

In [ ]:
def run():
    "Test linux router"
    topo = NetworkTopo()
    net = Mininet( topo=topo )  # controller is used by s1-s3
    net.start()
    info( '*** Routing Table on Router:\n' )
    info( net[ 'r0' ].cmd( 'route' ) )
    info( net[ 'r1' ].cmd( 'route' ) )
    CLI( net )
    net.stop()

if __name__ == '__main__':
    setLogLevel( 'info' )
    run()

*** Creating network
*** Adding controller
*** Adding hosts:
r0 r1 
*** Adding switches:
s1 s2 s3 
*** Adding links:
(s1, r0) 
*** Configuring hosts
r0 r1 *** defaultIntf: warning: r1 has no interfaces

*** Starting controller
c0 
*** Starting 3 switches
s1 s2 s3 ...
*** Routing Table on Router:
Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
192.168.1.0     *               255.255.255.0   U     0      0        0 r0-eth1
Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
*** Starting CLI:


mininet> links


s1-eth1<->r0-eth1 (OK OK) 


mininet> pingall


*** Ping: testing ping reachability
r0 -> X 
r1 -> X 
*** Results: 100% dropped (0/1 received)
